Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
import logging
from itertools import combinations
import pandas as pd
from tqdm.auto import tqdm

import random

import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [2]:
# Calculate the distance matrix

CITIES = pd.read_csv('../cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()


# US : 3_901_662_065_726,7 km shortees tour 
# italy : 4_172.76km shortest tour
# russia : 32_722.5km km shortest tour

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


In [3]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost





# First  Algorithm


In [4]:
best_tsp = None
best_cost = float('inf')

for i in range(len(CITIES)):


    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    city = i
    visited[city] = True
    tsp = [int(city)]

    while not np.all(visited):

        dist[:, city] = np.inf
        closest = np.argmin(dist[city])

        visited[closest] = True
        city = closest
        tsp.append(int(city))
    
    tsp.append(tsp[0])  # Chiudo  percorso
    cost = tsp_cost(tsp)

    
    if cost < best_cost:
        best_cost = cost
        best_tsp = tsp

# Output finale del miglior percorso
logging.info(f"Best path found has length {best_cost:.2f} km") 

for c1, c2 in zip(best_tsp, best_tsp[1:]):
    logging.info(
        f"step: {CITIES.at[c1,'name']} -> {CITIES.at[c2,'name']} ({DIST_MATRIX[c1,c2]:.2f}km)"
    )





INFO:root:Best path found has length 40051.59 km
INFO:root:step: Magadan -> Petropavlovsk‐Kamchatskiy (875.16km)
INFO:root:step: Petropavlovsk‐Kamchatskiy -> Yuzhno‐Sakhalinsk (1321.54km)
INFO:root:step: Yuzhno‐Sakhalinsk -> Komsomolsk‐na‐Amure (582.17km)
INFO:root:step: Komsomolsk‐na‐Amure -> Khabarovsk (274.21km)
INFO:root:step: Khabarovsk -> Ussuriysk (566.90km)
INFO:root:step: Ussuriysk -> Artyom (50.78km)
INFO:root:step: Artyom -> Vladivostok (34.21km)
INFO:root:step: Vladivostok -> Nakhodka (87.36km)
INFO:root:step: Nakhodka -> Blagoveshchensk (922.98km)
INFO:root:step: Blagoveshchensk -> Chita (1002.31km)
INFO:root:step: Chita -> Ulan‐Ude (402.25km)
INFO:root:step: Ulan‐Ude -> Irkutsk (238.28km)
INFO:root:step: Irkutsk -> Angarsk (34.88km)
INFO:root:step: Angarsk -> Bratsk (439.00km)
INFO:root:step: Bratsk -> Krasnoyarsk (538.01km)
INFO:root:step: Krasnoyarsk -> Achinsk (161.71km)
INFO:root:step: Achinsk -> Abakan (291.13km)
INFO:root:step: Abakan -> Novokuznetsk (285.72km)
INFO

# Second Algorithm


In [5]:
def fitness(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost



#  mutazione 
def insert_mutation(path):
    path = path.copy()
    i, j = random.sample(range(1, len(path) - 1), 2 )   # Evita i punti iniziale e finale

    # Se `j` è maggiore di `i`, l'elemento alla posizione `j` viene inserito dopo `i`
    if i < j:
        element = path.pop(j)
        path.insert(i + 1, element)
    # Se `i` è maggiore di `j`, l'elemento `j` viene inserito prima di `i`
    else:
        element = path.pop(j)
        path.insert(i, element)

    return path



def insert_mutation_temp1(path, temperature):
    path = path.copy()
    
    i = random.randint(1, len(path) ) # non ho messo -2 
    
    # Calcola una distanza massima tra `i` e `j` basata sulla temperatura
    max_distance = int((temperature / 1000) * (len(path) - 2)) 
    max_distance = max(1, min(max_distance, len(path) - 2))  # Limita max_distance all'intervallo valido


    # Scegli `j` con una distanza casuale fino a `max_distance` da `i`
    if random.random() < 0.5:
        # Scegli `j` a sinistra di `i`
        j = max(1, i - random.randint(1, max_distance))
    else:
        # Scegli `j` a destra di `i`
        j = min(len(path) - 2, i + random.randint(1, max_distance))

    # Esegui la mutazione: sposta l'elemento in `j` accanto a `i`
    if i < j:
        element = path.pop(j)
        path.insert(i + 1, element)
    else:
        element = path.pop(j)
        path.insert(i, element)

    return path




def insert_mutation_temp2(path, temperature):
    path = path.copy()
    
    # Scegli un indice casuale `i`, evitando il primo e l'ultimo per mantenere il percorso chiuso
    i = random.randint(1, len(path) - 2)
    
    # Calcola una distanza massima tra `i` e `j` basata sulla temperatura
    max_distance = int((temperature / 1000) * (len(path) - 2))
    max_distance = max(1, min(max_distance, len(path) - 2))  # Limita max_distance all'intervallo valido

    # Scegli `j` con una distanza casuale fino a `max_distance` da `i`
    if random.random() < 0.5:
        # Scegli `j` a sinistra di `i`
        j = max(1, i - random.randint(1, max_distance))
    else:
        # Scegli `j` a destra di `i`
        j = min(len(path) - 2, i + random.randint(1, max_distance))

    # Esegui la mutazione: sposta l'elemento in `j` accanto a `i`
    if i < j:
        element = path.pop(j)
        path.insert(i + 1, element)
    else:
        element = path.pop(j)
        path.insert(i, element)

    return path 






######
def scrambling_mutation(path):
    path = path.copy()
    # subset da mischiare
    i, j = sorted(random.sample(range(1, len(path) - 1), 2))  # Evita punti iniziale e finale

    subset = path[i:j+1]
    random.shuffle(subset)

    path[i:j+1] = subset

    return path



# recombination non sos
# inver / over   -> it's nice





In [6]:
## Simulated Annealing 

initial_temp = 1000  
cooling_rate = 0.995
num_iterations = 500_000  # iterazioni

# italy and RUSSIA  only :
buffer_size = 4  # russia - italy  
#buffer_size = int(0.02 * len(CITIES))    # ( US - CHINA ) # 2% del numero totale di città 


current_path = list(range(len(CITIES))) + [0]    # Percorso iniziale che parte e torna alla città 0   *** DA SISTEMARE  . inizia con un percorso meglio?????? 


best_path = current_path
current_cost = fitness(current_path)
best_cost = current_cost

temperature = initial_temp
cost_buffer = []

for iteration in tqdm (range(num_iterations) ): 
        


    #new_path = scrambling_mutation(current_path)  ## 
    new_path = insert_mutation(current_path)  ## 
    #new_path = insert_mutation_temp2(current_path, temperature)  ##

    new_cost = fitness(new_path)

    # Calcola la differenza di costo
    delta_cost = new_cost - current_cost

    # Decide se accettare il nuovo percorso
    if delta_cost < 0 or np.random.random() < np.exp(-delta_cost / temperature):
        current_path = new_path
        current_cost = new_cost

        if current_cost < best_cost:
            best_cost = current_cost
            best_path = current_path

    
    cost_buffer.append(current_cost)
    if len(cost_buffer) > buffer_size:
        cost_buffer.pop(0)

  # Aggiorna la temperatura solo se il trend è in discesa
    if iteration % buffer_size == 0 and len(cost_buffer) == buffer_size:
        avg_cost = np.mean(cost_buffer)
        if avg_cost < current_cost: # Se la media è in discesa ... 
            temperature *= cooling_rate


    if iteration % 10000 == 0:
        logging.info(f"Iteration {iteration}, Best Cost: {best_cost:.2f} km, Temperature: {temperature:.2f}")

    if temperature < 1e-5:
        break

logging.info(f"Best path found with length {best_cost:.2f} km,  after {iteration} iterations")




  0%|          | 0/500000 [00:00<?, ?it/s]

INFO:root:Iteration 0, Best Cost: 335828.72 km, Temperature: 1000.00
INFO:root:Iteration 10000, Best Cost: 78401.17 km, Temperature: 114.70
INFO:root:Iteration 20000, Best Cost: 54293.02 km, Temperature: 58.89
INFO:root:Iteration 30000, Best Cost: 50278.66 km, Temperature: 39.24
INFO:root:Iteration 40000, Best Cost: 45701.47 km, Temperature: 30.39
INFO:root:Iteration 50000, Best Cost: 43092.24 km, Temperature: 26.14
INFO:root:Iteration 60000, Best Cost: 42270.88 km, Temperature: 22.72
INFO:root:Iteration 70000, Best Cost: 40320.48 km, Temperature: 20.55
INFO:root:Iteration 80000, Best Cost: 39918.66 km, Temperature: 18.50
INFO:root:Iteration 90000, Best Cost: 39169.59 km, Temperature: 16.65
INFO:root:Iteration 100000, Best Cost: 38901.93 km, Temperature: 14.91
INFO:root:Iteration 110000, Best Cost: 38740.35 km, Temperature: 13.83
INFO:root:Iteration 120000, Best Cost: 38630.05 km, Temperature: 13.09
INFO:root:Iteration 130000, Best Cost: 38574.41 km, Temperature: 12.45
INFO:root:Iterat